In [1]:
from nltk import wsd
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from spacy.cli import download
from spacy import load
import warnings

nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')
nlp = load('en_core_web_sm')

! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # temp fix for lookup error.

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/wordnet2022.zip.


In [2]:
!pip install spacy


In [3]:
!pip install sentence_transformers

In [4]:
import pandas as pd
import torch  #framework to train model
from sentence_transformers import SentenceTransformer, util 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')


# Load model and initialize NLP components
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')  #LLM model ( 1 billion dataset) https://huggingface.co/sentence-transformers/all-mpnet-base-v2
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens 
                      if word.isalnum() and word not in stop_words]
    return " ".join(filtered_tokens)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Your vaccine FAQ data
vaccine_faq = [
    {
        "question": "Overview?",
        "answer": """Getting vaccinated and staying up to date with your COVID-19 vaccinations is the best way to remain protected against the most severe outcomes of COVID-19 infection including hospitalization and death. Vaccination may not always prevent symptomatic infection but it will help reduce severity if a person does become infected and decreases the risk of developing post COVID-19 condition (commonly known as long COVID).""",
        "source": "Ontario Ministry of Health - COVID-19 Vaccine Guidelines (2023)"
    },
    {
        "question": "Who can get vaccinated?",
        "answer": """COVID-19 vaccines are available to everybody aged 6 months and older in Ontario. The vaccine is available: 
- At no cost regardless of citizenship or immigration status 
- Even if you do not have an Ontario health card

If you do not have an Ontario health card, you may use another form of identification (photo identification is required if you are aged 18 or older) to verify your name and date of birth, such as:
- A driver’s licence or Ontario photo card
- A passport
- A piece of registered mail, pay stub, student card, library card
- Government issued identification from other jurisdictions (including foreign and expired government documents)""",
        "source": "Public Health Ontario - Special Populations Guidance"
    },
    {
        "question": "When you should get your next dose",
        "answer": """The ministry of health is recommending that everyone receive a COVID-19 vaccine this fall. 

Key guidelines:
1. Completed initial series: Wait 6 months (168 days) since last dose
2. Not completed initial series: Consult healthcare provider
3. Immunocompromised: Additional doses recommended (3-6 month intervals)
4. Post-COVID infection: Wait 2 months (56 days) after recovery

Special cases:
- Children 6 months-4 years: 2-dose series
- Age 5+ with no previous doses: Single dose
- Transplant/CAR-T therapy patients: Consult specialist

Always consult your healthcare provider for personalized timing.""",
        "source": "Health Canada - COVID-19 Vaccination Timing"
    },
    {
        "question": "Where you can get vaccinated?",
        "answer": """Available vaccination options:
1. Participating pharmacies
2. COVID‑19 vaccination portal
3. Provincial Vaccine Contact Centre (1-833-943-3900)
4. Public health unit clinics
5. Indigenous-led vaccination clinics
6. Primary care providers

Children 6 months-4 years: Check local public health unit websites for walk-in options.""",
        "source": "Ontario Vaccine Administration Protocol"
    },
    {
        "question": "When you should consult your doctor before getting the vaccine?",
        "answer": """Consult a healthcare provider if:
- Immunocompromised due to disease/treatment
- History of severe allergic reactions to vaccines
- Experienced myocarditis/pericarditis after previous dose
- Have bleeding disorders or take blood thinners
- Are pregnant or breastfeeding (for personalized advice)""",
        "source": "Ontario COVID-19 Vaccine Portal"
    }
]


In [6]:
# Preprocess and create embeddings
nltk.download('punkt_tab')
questions = [preprocess_text(q["question"]) for q in vaccine_faq]
answers = [q["answer"] for q in vaccine_faq]
sources = [q["source"] for q in vaccine_faq]
question_embeddings = model.encode(questions, convert_to_tensor=True)


[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# furqan running   y1 = [ 0 1]
# Talha eating     y2 = [0 0]

# What running      x = [1 1]

# #Cosine Similarity  |
# result for furqan = 0 + 1/root(2) = 1/root(2)
# result for Talha = 0 + 0/root(0)  = 0

In [8]:
def vaccine_chatbot(query, threshold=0.7):
    # Preprocess user query
    processed_query = preprocess_text(query)
    
    # Encode user query
    query_embedding = model.encode(processed_query, convert_to_tensor=True)
    
    # Calculate similarity scores
    cos_scores = util.pytorch_cos_sim(query_embedding, question_embeddings)[0]   #https://www.geeksforgeeks.org/cosine-similarity/
    
    # Find best match
    best_match_idx = torch.argmax(cos_scores).item()
    best_score = cos_scores[best_match_idx].item()
    
    if best_score >= threshold:
        response = {
            "answer": answers[best_match_idx],
            "source": sources[best_match_idx],
            "confidence": best_score
        }
    else:
        response = {
            "answer": "I'm sorry, I don't have information on that specific question. Please consult the Ontario COVID-19 portal or a healthcare professional for more details.",
            "source": "General Health Advisory",
            "confidence": best_score
        }
    
    # Format response
    formatted_response = f"Answer: {response['answer']}\n\nSource: {response['source']}\n(Confidence: {response['confidence']:.2f})"
    return formatted_response

In [9]:
# Example usage
questions = [
    "overview?",
    "Am I eligible for vaccination?",
    "How long should I wait after infection to get vaccinated?",
    "Where can I get my shot?",
    "Should I consult a doctor before vaccination?"
]

for q in questions:
    print(f"Q: {q}")
    print(vaccine_chatbot(q))
    print("\n" + "="*80 + "\n")


Q: overview?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer: Getting vaccinated and staying up to date with your COVID-19 vaccinations is the best way to remain protected against the most severe outcomes of COVID-19 infection including hospitalization and death. Vaccination may not always prevent symptomatic infection but it will help reduce severity if a person does become infected and decreases the risk of developing post COVID-19 condition (commonly known as long COVID).

Source: Ontario Ministry of Health - COVID-19 Vaccine Guidelines (2023)
(Confidence: 1.00)


Q: Am I eligible for vaccination?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer: I'm sorry, I don't have information on that specific question. Please consult the Ontario COVID-19 portal or a healthcare professional for more details.

Source: General Health Advisory
(Confidence: 0.65)


Q: How long should I wait after infection to get vaccinated?


...

Answer: I'm sorry, I don't have information on that specific question. Please consult the Ontario COVID-19 portal or a healthcare professional for more details.

Source: General Health Advisory
(Confidence: 0.63)


Q: Where can I get my shot?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer: I'm sorry, I don't have information on that specific question. Please consult the Ontario COVID-19 portal or a healthcare professional for more details.

Source: General Health Advisory
(Confidence: 0.50)


Q: Should I consult a doctor before vaccination?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Answer: Consult a healthcare provider if:
- Immunocompromised due to disease/treatment
- History of severe allergic reactions to vaccines
- Experienced myocarditis/pericarditis after previous dose
- Have bleeding disorders or take blood thinners
- Are pregnant or breastfeeding (for personalized advice)

Source: Ontario COVID-19 Vaccine Portal
(Confidence: 0.93)




In [10]:
# #Custom Chat
# def chat():
#     print("Chatbot: Hello! Ask me anything or type 'quit' to exit.")
#     while True:
#         user_input = input("You: ")
#         if user_input.lower() in ['quit', 'exit', 'bye']:
#             print("Chatbot: Goodbye!")
#             break
#         bot_response = vaccine_chatbot(user_input)
#         print(f"Chatbot: {bot_response}")

# # Start the chat
# chat()

In [15]:
# import tkinter as tk
# from tkinter import scrolledtext

# def vaccine_chat(user_input):
#     responses = {
#         "hello": "Hi there! How can I help you?",
#         "hi": "Hello! What would you like to know?",
#         "covid vaccine": "COVID vaccines are effective in preventing severe illness. Do you want details about any specific vaccine?",
#         "bye": "Goodbye! Stay safe!",
#     }
#     return responses.get(user_input.lower(), vaccine_chatbot(user_input))  # Handle unknown inputs safely

# # Function to process user input
# def send_message():
#     user_input = entry.get().strip()
#     if user_input:
#         chat_box.insert(tk.END, f"You: {user_input}\n", "user")
#         response = vaccine_chat(user_input)
#         chat_box.insert(tk.END, f"Chatbot: {response}\n", "bot")
#         chat_box.yview(tk.END)  # Auto-scroll to latest message
#         entry.delete(0, tk.END)

# # Create the main window
# root = tk.Tk()
# root.title("Smart Chatbot")
# root.configure(bg="#1e1e1e")

# # Chat history area
# chat_box = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=50, height=15, font=("Arial", 12), bg="#2d2d2d", fg="white")
# chat_box.tag_config("user", foreground="#7FDBFF")
# chat_box.tag_config("bot", foreground="#FFDC00")
# chat_box.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

# # User input field
# entry = tk.Entry(root, width=40, font=("Arial", 12), bg="#3c3f41", fg="white")
# entry.grid(row=1, column=0, padx=10, pady=10, sticky="w")

# # Send button
# send_button = tk.Button(root, text="Send", font=("Arial", 12, "bold"), bg="#FF851B", fg="white", command=send_message)
# send_button.grid(row=1, column=1, padx=10, pady=10, sticky="e")

# # Run the application
# root.mainloop()

In [16]:
import ipywidgets as widgets
from IPython.display import display

# Define chatbot response function
def vaccine_chat(user_input):
    responses = {
        "hello": "Hi there! How can I help you?",
        "hi": "Hello! What would you like to know?",
        "covid vaccine": "COVID vaccines are effective in preventing severe illness. Do you want details about any specific vaccine?",
        "bye": "Goodbye! Stay safe!",
    }
    return responses.get(user_input.lower(), "I'm sorry, I don't understand. Can you ask something else?")

# Create chatbot UI
chat_output = widgets.Output()
user_input = widgets.Text(placeholder="Type your question here...")

# Handle user input
def on_submit(sender):
    with chat_output:
        print(f"You: {user_input.value}")
        print(f"Chatbot: {vaccine_chat(user_input.value)}")
    user_input.value = ""  # Clear input field

user_input.on_submit(on_submit)

# Display chatbot UI in Kaggle
display(user_input, chat_output)


<ipython-input-16-2ff0bf096fba>:25: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  user_input.on_submit(on_submit)


Text(value='', placeholder='Type your question here...')

Output()